# Лабораторная работа 4. Часть 1. Ранжирование.



Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 7 баллов. Сдавать задание после указанного в lk срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).



## Знакомство с данными

### Ранжирование организаций по пользовательскому запросу

Что мы обычно делаем, когда нам нужно найти определённое место, но не знаем его местоположения? Используем поиск на картах.

В этой лабораторной работе вам будет необходимо построить небольшую поисковую систему, позволяющую отранжировать организации по запросу пользователя.

Для обучения вам даны 2000 запросов и более 13 тысяч найденных по ним организаций. Для каждой пары "запрос — организация" была посчитана релевантность, по которой и происходит ранжирование.

**(2 балла) Задание 1.** Загрузите [данные](https://disk.yandex.ru/d/Bf3P4H8FDYe7-g) о запросах и их релевантности (*train.csv*), а также информацию об организациях (*train_org_information.json*) и рубриках (*train_rubric_information.json*)

Для дальнейших экспериментов необходимо посчитать небольшой набор базовых факторов. С использованием информации о запросах и организациях, посчитайте факторы, которые на ваш взгляд будут полезными для предсказания релевантности.

Примерами текстовых факторов могут служить:
 - кол-во слов в запросе и названии организации;
 - пословные/N-граммные пересечения слов запроса и названия организации (также можно использовать синонимы названия организации и адрес организации): кол-во слов в пересечении, [мера Жаккара](https://en.wikipedia.org/wiki/Jaccard_index) и пр.;
 - кол-во различных синонимичных названий организации (поле *names* в описании организации);
 - One-hot-encoded язык запроса.
 
По информации о географическом положении:
 - факт совпадения региона, где задавался запрос и региона организации;
 - координаты показанной области;
 - размеры показанной области;
 - меры, характеризующие близость координат организации к показанному окну: расстояние до центра области и другие.
 
Факторы, описывающие организацию:
 - one-hot-encoding фактор cтраны или региона организации (важно: не используйте one-hot-encoding факторы, в которых больше 10 значений; если в факторе слишком много значений, ограничьтесь, например, только самыми популярными категориями)
 - кол-во рабочих дней в неделе и общая продолжительность работы (поле *work_intervals* в описании организации)
 - кол-во рубрик (поле *rubrics* в описании организации)
 
![](https://miro.medium.com/max/1500/0*FwubnnoNlt6Coo9j.png)

В этом задании не нужно использовать многомерные представления текстовой информации (tfidf и прочие embeddings) и информацию о кликах (*train_clicks_information.json*). Придумывать сверхсложные факторы тоже необязательно.

Вы можете реализовать описанные выше факторы и/или придумать свои. Но зачастую такие простые признаки могут приносить наибольшую пользу.

В итоге у вас должно получиться от 15 до 50 факторов, характеризующих запрос и организацию и покрывающих основные источники данных (кроме кликов). Это наш основной датасет, который будет использоваться в экспериментах.



In [1]:
import json
import langdetect
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_data = pd.read_csv('./train.csv')
train_data.head(5)

,query_id,query,region,org_name,org_id,window_center,window_size,relevance
0,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Суд Жовтневого району міста Дніпропетровськ,1021049127,"34.613119,48.506531","0.025928,0.017380",0.0
1,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Дніпропетровський окружний адміністративний суд,1602348889,"34.613119,48.506531","0.025928,0.017380",0.0
2,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Бабушкінський районний суд,1105837793,"34.613119,48.506531","0.025928,0.017380",0.0
3,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Красногвардійський районний суд,1066267658,"34.613119,48.506531","0.025928,0.017380",0.0
4,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Жовтневий суд,1661586235,"34.613119,48.506531","0.025928,0.017380",0.0


In [3]:
print("Число строк: {}\nЧисло столбцов: {}".format(*train_data.shape))

Число строк: 29274
Число столбцов: 8


In [4]:
unique_query_id = np.unique(train_data.query_id.values)
print("Число уникальных query_id:", len(unique_query_id))

Число уникальных query_id: 2000


Посмотрим на несколько строк в train

In [5]:
for i in [11, 124, 200]:
    for col, data in zip(train_data.columns, train_data.iloc[i].values):
        print(col, ":", data)
    print()

query_id : 11
query : суд, Украина, Днепропетровская область, Днепродзержинский городской совет
region : 21775
org_name : Дніпропетровський окружний адміністративний суд
org_id : 1380543593
window_center : 34.613119,48.506531
window_size : 0.025928,0.017380
relevance : 0.0

query_id : 94
query : банкоматы
region : 143
org_name : Надра Банк, банкомат
org_id : 1207964833
window_center : 30.636340,50.422901
window_size : 0.098170,0.036736
relevance : 0.14

query_id : 116
query : Аптеки
region : 143
org_name : Аптека Фармація
org_id : 1180197860
window_center : 30.417718,50.428612
window_size : 0.025748,0.026062
relevance : 0.14



А теперь посмотрим на query_id=11

In [6]:
for _, row in train_data[train_data.query_id == 11].iterrows():
    print("query:", row.query)
    print("org_name", row.org_name)
    print("relevance", row.relevance)
    print()

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Суд Жовтневого району міста Дніпропетровськ
relevance 0.0

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Дніпропетровський окружний адміністративний суд
relevance 0.0

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Бабушкінський районний суд
relevance 0.0

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Красногвардійський районний суд
relevance 0.0

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Жовтневий суд
relevance 0.0

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Ленінський районний суд міста Дніпропетровськ
relevance 0.0

query: суд, Украина, Днепропетровская область, Днепродзержинский городской совет
org_name Апеляційний суд Дніпропетровської області
relevance 0.0

query: суд, 

In [7]:
path_to_org = './train_org_information.json'
org_data = pd.read_json(path_to_org, orient='index', convert_axes=False, convert_dates=False)
org_data.head()

,names,rubrics,work_intervals,address
1255014404,"[{'value': {'locale': 'en', 'value': 'Bely Bok...","[20277, 20679, 21237]","[{'time_minutes_begin': 540, 'day': 'everyday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
1111883782,"[{'value': {'locale': 'en', 'value': 'Vyborgsk...",[30723],"[{'time_minutes_begin': 510, 'day': 'saturday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
39713767434,"[{'value': {'locale': 'en', 'value': 'Sberbank...",[30348],"[{'time_minutes_begin': 570, 'day': 'saturday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
1336016908,"[{'value': {'locale': 'en', 'value': 'Filial b...","[30057, 30111, 30114]","[{'time_minutes_begin': 480, 'day': 'saturday'...","{'region_code': 'RU', 'formatted': {'locale': ..."
149331116064,"[{'value': {'locale': 'ru', 'value': 'Школа'}}]",[30723],[],"{'region_code': 'RU', 'formatted': {'locale': ..."


In [8]:
with open(path_to_org, 'r') as read_file:
    train_org_json = json.load(read_file)

In [9]:
org_data.names[0]

[{'value': {'locale': 'en', 'value': 'Bely Boks'}},
 {'value': {'locale': 'ru', 'value': 'Белый бокс'}}]

In [10]:
org_data.work_intervals[0]

[{'time_minutes_begin': 540, 'day': 'everyday', 'time_minutes_end': 1260}]

In [11]:
org_data.address[0]

{'region_code': 'RU',
 'formatted': {'locale': 'ru',
  'value': 'Россия, Санкт-Петербург, Кронштадтская улица, 19'},
 'pos': {'type': 'Point', 'coordinates': [30.257127, 59.865881]},
 'geo_id': 2}

In [12]:
path_to_rubric = './train_rubric_information.json'
rubric_data = pd.read_json(path_to_rubric, orient='index', convert_axes=False, convert_dates=False)
rubric_data.head()

,keywords,phrases,descriptions,names
273147906,"[{'locale': 'ru', 'value': 'велосипед парковка...","[{'locale': 'de', 'value': 'Parkplatz für Fahr...","[{'value': {'locale': 'tr', 'value': 'Bisiklet...","[{'locale': 'en', 'value': 'Bicycle stand'}, {..."
30723,"[{'locale': 'tr', 'value': 'liseler, ortaokul,...","[{'locale': 'de', 'value': 'Bildungseinrichtun...","[{'value': {'locale': 'tr', 'value': 'Çocuklar...","[{'locale': 'de', 'value': 'Allgemeinbildende ..."
30724,"[{'locale': 'tr', 'value': 'arastırma, bilim, ...","[{'locale': 'tr', 'value': 'Bilim Arastırma En...","[{'value': {'locale': 'tr', 'value': 'Bilim ar...","[{'locale': 'tr', 'value': 'Bilim Araştırma En..."
30725,"[{'locale': 'tr', 'value': 'eğitim kontrol'}]","[{'locale': 'en', 'value': 'Board of Education...","[{'value': {'locale': 'tr', 'value': 'Eğitim k...","[{'locale': 'it', 'value': 'Amministrazione de..."
30731,"[{'locale': 'ru', 'value': 'Новые технологии -...","[{'locale': 'en', 'value': 'implementation of ...","[{'value': {'locale': 'tr', 'value': 'Nanotekn...","[{'locale': 'de', 'value': 'Innovative Technol..."


In [13]:
with open(path_to_rubric, 'r') as read_file:
    train_rubric_json = json.load(read_file)

In [14]:
rubric_data.keywords[1]

[{'locale': 'tr',
  'value': 'liseler, ortaokul, liseleri, özel, ilköğretim, okul, orta, kurumları, ilkokul, düz, öğretim, kurumlari, fen, okullar, okulları, eğitim, ogretim, genel, okulu, ortaokullar, ilk'},
 {'locale': 'ru',
  'value': 'Школы общеобразовательные средняя экстернат вечерняя сш начальная детская'}]

In [15]:
rubric_data.phrases[1]

[{'locale': 'de', 'value': 'Bildungseinrichtung'},
 {'locale': 'de', 'value': 'Gesamtschulen'},
 {'locale': 'de', 'value': 'Grundschule'},
 {'locale': 'en', 'value': 'K-12 education'},
 {'locale': 'en', 'value': 'K-12 school'},
 {'locale': 'de', 'value': 'Lehranstalt'},
 {'locale': 'en', 'value': 'Middle Schools'},
 {'locale': 'de', 'value': 'Mittelschule'},
 {'locale': 'tr', 'value': 'Ortaokul'},
 {'locale': 'en', 'value': 'Primary Secondary Schools'},
 {'locale': 'en', 'value': 'Public Schools'},
 {'locale': 'de', 'value': 'Schule'},
 {'locale': 'de', 'value': 'Unterrichtsanstalt'},
 {'locale': 'de', 'value': 'allgemein bildende Schulen'},
 {'locale': 'en', 'value': "children's school"},
 {'locale': 'en', 'value': 'distance education'},
 {'locale': 'tr', 'value': 'düz liseler'},
 {'locale': 'fr', 'value': 'enseignement secondaire'},
 {'locale': 'en', 'value': 'evening schools'},
 {'locale': 'fr', 'value': 'externat'},
 {'locale': 'tr', 'value': 'fen liseleri'},
 {'locale': 'tr', 'val

In [16]:
rubric_data.descriptions[1]

[{'value': {'locale': 'tr',
   'value': 'Çocuklara orta eğitimi veren eğitim kurumları. Adı "Eğitim merkezi" olarak belirtilmişse kurumun ortaokul mu anaokulu mu olduğunu sormak gerek. Hem ortaokul hem de anaokuluysa her iki kategori seçmek gerek.\n\nBenzer kategoriler: Lise, Kolejler, Liseler, Sanatoryum okulu, Özel Okullar, Yatılı okul.'}},
 {'value': {'locale': 'ru',
   'value': 'Образовательные учреждения, дающие общее среднее образование детям. Если в названии указано Центр образования, то нужно уточнить, школа это или детский сад. Если и школа, и детский сад, ставим обе рубрики. Название оформляется следующим образом: Центр Образования № 1811; Центр Образования № 1811 Начальная школа; Центр Образования № 1811 Дошкольное отделение.\n\nПохожие рубрики: Гимназия, Колледж, Лицей, Школа санаторного типа, Частная школа, Школа-интернат.'}}]

In [17]:
rubric_data.names[1]

[{'locale': 'de', 'value': 'Allgemeinbildende Schule'},
 {'locale': 'tr', 'value': 'Ortaokullar'},
 {'locale': 'en', 'value': 'School'},
 {'locale': 'it', 'value': "Scuola d'insegnamento generale"},
 {'locale': 'fr', 'value': 'École secondaire'},
 {'locale': 'kk', 'value': 'Жалпы білім беретін мектеп'},
 {'locale': 'uk', 'value': 'Загальноосвітня школа'},
 {'locale': 'ru', 'value': 'Общеобразовательная школа'},
 {'locale': 'uz', 'value': 'Умумтаълим мактаби'},
 {'locale': 'hy', 'value': 'Հանրակրթական դպրոց'},
 {'locale': 'fa', 'value': 'مدرسه جامع'}]

In [18]:
def getWordsFromString(x) -> str:
    return re.sub(r"[^A-Za-zА-Яа-я]+", ' ', x).lower().split()

def getIntersectionWordsCount(x) -> int:
    return len(set(getWordsFromString(x[0])) & set(getWordsFromString(x[1])))

def getSynonymousNamesCount(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        return len(node["names"])
    return 0

def getEncodedLangs():
    langs = train_data['query'].apply(lambda x: langdetect.detect(x))
    sortedLangs = [lang for lang, _ in Counter(langs).most_common()]
    langsEncoder = OneHotEncoder(sparse=False).fit(langs.values.reshape(-1,1))
    encodedLangs = langsEncoder.transform(langs.values.reshape(-1,1))
    langsIdx = [np.where(langsEncoder.categories_[0] == lang)[0][0] for lang in sortedLangs[:10]]
    
    newEncodedLangs = np.zeros((encodedLangs.shape[0], 11))
    for i, idx in enumerate(langsIdx):
        newEncodedLangs[:, i] = encodedLangs[:, idx]
    for row in newEncodedLangs:
        if row.sum() == 0:
            row[-1] = 1
    return newEncodedLangs

def getRubricsCount(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        return len(node["rubrics"])
    return 0

def getWorkTime(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        work_interval_sum = 0
        for work_interval in node["work_intervals"]:
            work_interval_sum += work_interval["time_minutes_end"] - work_interval["time_minutes_begin"]
        if len(node["work_intervals"]) != 0:
            return work_interval_sum / len(node["work_intervals"])
    return 0

def getDaysCount(x):
    node = train_org_json.get(str(x))
    if node is not None:
        daysCount = 0
        for work_interval in node["work_intervals"]:
            day = work_interval["day"]
            if day == "everyday":
                daysCount = max(daysCount, 7)
            elif day == "weekdays":
                daysCount = max(daysCount, 5)
            elif day == "weekend":
                daysCount = max(daysCount, 2)
            elif day in ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]:
                daysCount = 1
            else:
                daysCount = 0
        return daysCount
    return 0

def getRegionCode(x) -> str:
    node = train_org_json.get(str(x))
    if node is not None:
        return node["address"]["region_code"]
    return 0

def getEncodedRegionCode():
    regionCodes = train_data["org_id"].apply(getRegionCode).values
    sortedRegionCodes = [code for code, _ in Counter(regionCodes).most_common()[:10]]
    codesEncoder = OneHotEncoder(sparse=False).fit(regionCodes.reshape(-1,1))
    encodedCodes = codesEncoder.transform(regionCodes.reshape(-1,1))
    codesIdx = [np.where(codesEncoder.categories_[0] == code)[0][0] for code in sortedRegionCodes[:10]]
    
    newEncodedCodes = np.zeros((encodedCodes.shape[0], 11))
    for i, idx in enumerate(codesIdx):
        newEncodedCodes[:, i] = encodedCodes[:, idx]
    for row in newEncodedCodes:
        if row.sum() == 0:
            row[-1] = 1
    return newEncodedCodes

In [19]:
df = pd.DataFrame()
df['query_id'] = train_data['query_id']
df['query_words_count'] = train_data['query'].apply(lambda query: len(getWordsFromString(query)))
df['org_name_words_count'] = train_data['org_name'].apply(lambda name: len(getWordsFromString(name)))
df['intersection_words_count'] = train_data[['query', 'org_name']].apply(getIntersectionWordsCount, axis=1)
df['jaccard_score'] = df['intersection_words_count'] / (df['query_words_count'] + df['org_name_words_count'] - df['intersection_words_count'])
df['synonymous_names_count'] = train_data['org_id'].apply(getSynonymousNamesCount)

In [20]:
df['rubrics_count'] = train_data['org_id'].apply(getRubricsCount)
df['worktime'] = train_data['org_id'].apply(getWorkTime)
df['workdays_count'] = train_data['org_id'].apply(getDaysCount)

In [21]:
df["relevance"] = train_data["relevance"]

In [22]:
df.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count,relevance
0,11,7,7,1,0.076923,5,1,240.0,1,0.0
1,11,7,7,1,0.076923,5,1,540.0,5,0.0
2,11,7,4,1,0.100000,7,1,480.0,5,0.0
3,11,7,4,1,0.100000,7,1,240.0,1,0.0
4,11,7,2,1,0.125000,3,1,540.0,5,0.0


## Ранжирование

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.
3. **Listwise подход**. Этот подход использует все объекты для вычисления функции потерь, стараясь явно оптимизировать правильный порядок.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

В рамках нашей задачи «документом» будет являться организация.

Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание, что разбивать необходимо множество запросов, а не строчки датасета.

In [23]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(test_size=.30, n_splits=1, random_state = 457).split(df, groups=df['query_id'])

X_train_inds, X_test_inds = next(gss)

new_train_data = df.iloc[X_train_inds]
X_train = new_train_data.loc[:, ~new_train_data.columns.isin(['relevance'])]
y_train = new_train_data.loc[:, new_train_data.columns.isin(['query_id', 'relevance'])]

groups = train_data.groupby('query_id').size().to_frame('size')['size'].to_numpy()

new_test_data = df.iloc[X_test_inds]

X_test = new_test_data.loc[:, ~new_test_data.columns.isin(['relevance'])]
y_test = new_test_data.loc[:, new_test_data.columns.isin(['query_id', 'relevance'])]

In [24]:
X_train.shape, X_test.shape

((20510, 9), (8764, 9))

In [25]:
X_train.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count
0,11,7,7,1,0.076923,5,1,240.0,1
1,11,7,7,1,0.076923,5,1,540.0,5
2,11,7,4,1,0.100000,7,1,480.0,5
3,11,7,4,1,0.100000,7,1,240.0,1
4,11,7,2,1,0.125000,3,1,540.0,5


In [26]:
y_train.head()

,query_id,relevance
0,11,0.0
1,11,0.0
2,11,0.0
3,11,0.0
4,11,0.0


In [27]:
X_test.head(1)

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count
147,116,1,4,0,0.0,6,1,780.0,7


In [28]:
y_test.head(1)

,query_id,relevance
147,116,0.14


Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам, где разбиение должно быть по запросам, а не строчкам датасета.

###  Ранжируем с XGBoost и CatBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

Реализованные в CatBoost ранжирующие функции потерь можной найти [здесь](https://catboost.ai/docs/concepts/loss-functions-ranking.html#groupwise-metrics).

**(3 балла) Задание 2.** Попробуйте различные функции потерь (регрессионные и ранжирующие) для моделей XGBoost и CatBoost. Настройте основные параметры моделей (глубина, кол-во деревьев, глубина, скорость обучения, регуляризация).  
Сравните построенные модели с точки зрения метрики nDCG на контроле и проанализируйте полученные результаты:
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?

1) Перед обучением разобьем обучающую выборку на 5 фолдов для подбора параметров моделей

In [29]:
n_folds = 5
val_gss = GroupShuffleSplit(n_splits=n_folds, test_size=0.2, random_state=457)

fold_indexes = []
for train_idx, test_idx in val_gss.split(new_train_data, groups=new_train_data['query_id']):
    fold_indexes.append({'train_idx': train_idx, 'test_idx': test_idx})

In [30]:
fold_indexes

[{'train_idx': array([    0,     1,     2, ..., 20507, 20508, 20509]),
  'test_idx': array([  250,   251,   252, ..., 20495, 20496, 20497])},
 {'train_idx': array([    0,     1,     2, ..., 20507, 20508, 20509]),
  'test_idx': array([   91,    92,    93, ..., 20495, 20496, 20497])},
 {'train_idx': array([    0,     1,     2, ..., 20495, 20496, 20497]),
  'test_idx': array([   91,    92,    93, ..., 20507, 20508, 20509])},
 {'train_idx': array([    0,     1,     2, ..., 20507, 20508, 20509]),
  'test_idx': array([   20,    21,    22, ..., 20345, 20346, 20347])},
 {'train_idx': array([   20,    21,    22, ..., 20507, 20508, 20509]),
  'test_idx': array([    0,     1,     2, ..., 20345, 20346, 20347])}]

In [31]:
def get_i_fold(X, i):
    X_val_train = X.iloc[fold_indexes[i]['train_idx']].loc[:, ~X.columns.isin(['relevance'])]
    y_val_train = X.iloc[fold_indexes[i]['train_idx']].loc[:, X.columns.isin(['query_id','relevance'])]
    
    X_val_test = X.iloc[fold_indexes[i]['test_idx']].loc[:, ~X.columns.isin(['relevance'])]
    y_val_test = X.iloc[fold_indexes[i]['test_idx']].loc[:, X.columns.isin(['query_id','relevance'])]
    return (X_val_train, y_val_train), (X_val_test, y_val_test)

In [32]:
(X_val_0, y_val_0), (X_val_1, y_val_1) = get_i_fold(new_train_data, 0)

In [33]:
X_val_0.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count
0,11,7,7,1,0.076923,5,1,240.0,1
1,11,7,7,1,0.076923,5,1,540.0,5
2,11,7,4,1,0.100000,7,1,480.0,5
3,11,7,4,1,0.100000,7,1,240.0,1
4,11,7,2,1,0.125000,3,1,540.0,5


In [34]:
y_val_0.head()

,query_id,relevance
0,11,0.0
1,11,0.0
2,11,0.0
3,11,0.0
4,11,0.0


In [35]:
X_val_1.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count
362,216,1,2,1,0.5,6,1,1440.0,7
363,216,1,2,1,0.5,5,1,0.0,0
364,216,1,2,1,0.5,5,1,1440.0,7
365,216,1,2,1,0.5,5,1,1440.0,7
366,216,1,2,1,0.5,6,1,1440.0,7


In [36]:
y_val_1.head()

,query_id,relevance
362,216,0.14
363,216,0.14
364,216,0.14
365,216,0.14
366,216,0.14


2) Напишем функцию, вычисляющую метрику nDCG

In [37]:
def DCG(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2) + 1e-6
    return np.sum(gains / discounts)

In [38]:
def nDCG(y_true, y_pred, k=10):
    best = DCG(y_true, y_true, k) + 1e-6
    actual = DCG(y_true, y_pred, k)
    return actual / best

In [39]:
def nDCG_query(y_true_group, y_pred_group):
    scores = []
    for query_id in y_pred_group.index.values:
        scores.append(nDCG(y_true_group[query_id], y_pred_group[query_id]))
    return np.mean(scores)

3) Напишем функцию, которая делает кросс-валидацию

In [40]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['query_id'])])

In [41]:
def cross_val_scores(model):
    ndcg_score = []
    for i in range(n_folds):
        (X_val_train, y_val_train), (X_val_test, y_val_test) = get_i_fold(new_train_data, i)
        model.fit(X_val_train.loc[:, ~X_val_train.columns.isin(['query_id'])], y_val_train.relevance, verbose=True)

        predictions = (X_val_test.groupby('query_id').apply(lambda x: predict(model, x)))
        ndcg_score.append(nDCG_query(y_val_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions))
    return ndcg_score

### XGBoost

In [42]:
import xgboost as xgb

In [44]:
X_train.loc[:, ~X_train.columns.isin(['query_id'])].head(1)

,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count
0,7,7,1,0.076923,5,1,240.0,1


In [45]:
y_train.head(1)

,query_id,relevance
0,11,0.0


#### reg:linear

In [56]:
model = xgb.XGBRegressor(objective='reg:linear')
model.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])], y_train.relevance, verbose=0)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=32,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [57]:
predictions = (X_test.groupby('query_id').apply(lambda x: predict(model, x)))

In [58]:
predictions

query_id
116      [0.11655478, 0.11695577, 0.12280623, 0.1249737...
198      [0.084806465, 0.11102717, 0.12001192, 0.120767...
202      [0.118985526, 0.11377729, 0.14943244, 0.120745...
232      [0.11378162, 0.1161177, 0.119079664, 0.1300364...
259      [0.07688985, 0.11349801, 0.122818634, 0.11719,...
                               ...                        
35959    [0.008444955, 0.02298762, 0.0485556, 0.0084449...
36170    [0.052228518, 0.06538302, 0.020245967, 0.24301...
36282    [0.023292601, 0.039437298, 0.024750715, 0.0195...
36304    [0.06281152, 0.0698221, 0.078189336, 0.1415823...
36337    [0.081715316, 0.08090903, 0.06880772, 0.111690...
Length: 600, dtype: object

In [59]:
y_test.groupby('query_id').apply(lambda x: list(x.relevance))

query_id
116      [0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.1...
198      [0.07, 0.07, 0.07, 0.14, 0.14, 0.14, 0.14, 0.1...
202      [0.07, 0.07, 0.07, 0.07, 0.07, 0.14, 0.14, 0.1...
232      [0.0, 0.07, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14...
259      [0.0, 0.0, 0.07, 0.14, 0.14, 0.14, 0.14, 0.14,...
                               ...                        
35959    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
36170       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.45]
36282    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
36304    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
36337    [0.0, 0.0, 0.0, 0.01, 0.01, 0.02, 0.02, 0.04, ...
Length: 600, dtype: object

In [60]:
nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)

0.6768208329927655

А теперь начнем подбирать гиперпараметры по кросс-валидации

In [63]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8]
for depth in max_depths:
    model = xgb.XGBRegressor(
                objective='reg:linear',
                max_depth=depth,
                verbosity=0
            )
    
    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores(model))))

max_depth = 1, mean nDCG on cv = 0.6773071233403879
max_depth = 2, mean nDCG on cv = 0.6876887644200441
max_depth = 3, mean nDCG on cv = 0.6789186465196557
max_depth = 4, mean nDCG on cv = 0.675239519995399
max_depth = 5, mean nDCG on cv = 0.6718224501855453
max_depth = 6, mean nDCG on cv = 0.6697230091058549
max_depth = 7, mean nDCG on cv = 0.6600284378245252
max_depth = 8, mean nDCG on cv = 0.6595982976146908


In [69]:
#Скорость обучения
lrs = [1e-3, 1e-2, 1e-1, 1]
for lr in lrs:
    model = xgb.XGBRegressor(
                objective='reg:linear',
                learning_rate=lr,
                max_depth=2,
                verbosity=0
            )
    
    print("lr = {}, mean nDCG on cv = {}".format(lr, np.mean(cross_val_scores(model))))

lr = 0.001, mean nDCG on cv = 0.901595693660804
lr = 0.01, mean nDCG on cv = 0.8429928931952151
lr = 0.1, mean nDCG on cv = 0.6925664402847149
lr = 1, mean nDCG on cv = 0.6736485400355299


In [70]:
#Число деревьев
n_estimators = [10, 25, 50, 75, 100]
for n in n_estimators:
    model = xgb.XGBRegressor(
                objective='reg:linear',
                n_estimators=n,
                learning_rate=1e-3,
                max_depth=2,
                verbosity=0
            )
    
    print("n_estimators = {}, mean nDCG on cv = {}".format(n, np.mean(cross_val_scores(model))))

n_estimators = 10, mean nDCG on cv = 0.9028176034883838
n_estimators = 25, mean nDCG on cv = 0.9020395133143995
n_estimators = 50, mean nDCG on cv = 0.9020395133143995
n_estimators = 75, mean nDCG on cv = 0.901595693660804
n_estimators = 100, mean nDCG on cv = 0.901595693660804


In [73]:
#Регуляризация
reg_alpha = [1e-4, 1e-3, 1e-2, 1e-1, 1] # for L1 regularization
reg_lambda = [1e-4, 1e-3, 1e-2, 1e-1, 1] # for L2 regularization
for alpha in reg_alpha:
    for lamb in reg_lambda:
        model = xgb.XGBRegressor(
                    objective='reg:linear',
                    n_estimators=10,
                    learning_rate=1e-3, 
                    max_depth=2,
                    reg_alpha=alpha,
                    reg_lambda=lamb,
                    verbosity=0
                )
        
        print("alpha = {}, lambda = {}, mean nDCG on cv = {}".format(alpha, lamb, np.mean(cross_val_scores(model))))

alpha = 0.0001, lambda = 0.0001, mean nDCG on cv = 0.9020395133143995
alpha = 0.0001, lambda = 0.001, mean nDCG on cv = 0.9020395133143995
alpha = 0.0001, lambda = 0.01, mean nDCG on cv = 0.9020395133143995
alpha = 0.0001, lambda = 0.1, mean nDCG on cv = 0.9020395133143995
alpha = 0.0001, lambda = 1, mean nDCG on cv = 0.9028176034883838
alpha = 0.001, lambda = 0.0001, mean nDCG on cv = 0.9020395133143995
alpha = 0.001, lambda = 0.001, mean nDCG on cv = 0.9020395133143995
alpha = 0.001, lambda = 0.01, mean nDCG on cv = 0.9020395133143995
alpha = 0.001, lambda = 0.1, mean nDCG on cv = 0.9020395133143995
alpha = 0.001, lambda = 1, mean nDCG on cv = 0.9028176034883838
alpha = 0.01, lambda = 0.0001, mean nDCG on cv = 0.9020395133143995
alpha = 0.01, lambda = 0.001, mean nDCG on cv = 0.9020395133143995
alpha = 0.01, lambda = 0.01, mean nDCG on cv = 0.9020395133143995
alpha = 0.01, lambda = 0.1, mean nDCG on cv = 0.9020395133143995
alpha = 0.01, lambda = 1, mean nDCG on cv = 0.902817603488383

Получившиеся параметры: n_estimators=10, learning_rate=1e-3, max_depth=2, reg_alpha=1

In [76]:
xgb_linear = xgb.XGBRegressor(
                objective='reg:linear',
                n_estimators=n,
                learning_rate=1e-3, 
                max_depth=2,
                reg_alpha=1,
                verbosity=0
            )

xgb_linear.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])], y_train.relevance, verbose=0)

predictions = (X_test.groupby('query_id').apply(lambda x: predict(xgb_linear, x)))
xgb_linear_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG sore for XGBoost with reg:linear:', xgb_linear_ndcg)

nDCG sore for XGBoost with reg:linear: 0.8993894282924212


Попробуем ещё раз оптимизировать макс глубину

In [77]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8]
for depth in max_depths:
    model = xgb.XGBRegressor(
                objective='reg:linear',
                n_estimators=10,
                learning_rate=1e-3, 
                max_depth=depth,
                reg_alpha=1,
                verbosity=0
            )
    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores(model))))

max_depth = 1, mean nDCG on cv = 0.9060168475669513
max_depth = 2, mean nDCG on cv = 0.9038274646330962
max_depth = 3, mean nDCG on cv = 0.8734368758929337
max_depth = 4, mean nDCG on cv = 0.8730923119869953
max_depth = 5, mean nDCG on cv = 0.8731084344844907
max_depth = 6, mean nDCG on cv = 0.8731084344844907
max_depth = 7, mean nDCG on cv = 0.8731084344844907
max_depth = 8, mean nDCG on cv = 0.8731084344844907


In [82]:
xgb_linear = xgb.XGBRegressor(
                objective='reg:linear',
                n_estimators=10,
                learning_rate=1e-3, 
                max_depth=1,
                reg_alpha=1,
                verbosity=0
            )
xgb_linear.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])], y_train.relevance, verbose=0)

predictions = (X_test.groupby('query_id').apply(lambda x: predict(xgb_linear, x)))
xgb_linear_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG sore for XGBoost with reg:linear:', xgb_linear_ndcg)

nDCG sore for XGBoost with reg:linear: 0.9113383867102114


Огонь 🔥🔥🔥, получили больший скор с параметрами n_estimators=10, learning_rate=1e-3, max_depth=1, reg_alpha=1. Финальный nDCG для XGBoost с функцией потерь reg:linear равен 0.9113383867102114. Сделаем подбор гиперпараметров для остальных моделей

#### rank:pairwise

In [91]:
def cross_val_scores(model):
    ndcg_score = []
    for i in range(n_folds):
        (X_val_train, y_val_train), (X_val_test, y_val_test) = get_i_fold(new_train_data, i)
        model.fit(X_val_train.loc[:, ~X_val_train.columns.isin(['query_id'])],
                  y_val_train.relevance,
                  group=X_val_train.groupby('query_id').size().to_frame('size')['size'].to_numpy(),
                  verbose=True)

        predictions = (X_val_test.groupby('query_id').apply(lambda x: predict(model, x)))
        ndcg_score.append(nDCG_query(y_val_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions))
    return ndcg_score

In [92]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8]
for depth in max_depths:
    model = xgb.XGBRanker(
                objective='rank:pairwise',
                max_depth=depth
            )
    
    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores(model))))

max_depth = 1, mean nDCG on cv = 0.7318310095134807
max_depth = 2, mean nDCG on cv = 0.7236469307379579
max_depth = 3, mean nDCG on cv = 0.721403280135212
max_depth = 4, mean nDCG on cv = 0.7120017800867692
max_depth = 5, mean nDCG on cv = 0.6992698258062948
max_depth = 6, mean nDCG on cv = 0.6945165769062985
max_depth = 7, mean nDCG on cv = 0.6855018194556739
max_depth = 8, mean nDCG on cv = 0.6754270186166655


In [93]:
#Скорость обучения
lrs = [1e-3, 1e-2, 1e-1, 1]
for lr in lrs:
    model = xgb.XGBRanker(
                objective='rank:pairwise',
                learning_rate=lrб
                max_depth=1
            )
    
    print("lr = {}, mean nDCG on cv = {}".format(lr, np.mean(cross_val_scores(model))))

lr = 0.001, mean nDCG on cv = 0.805722399468667
lr = 0.01, mean nDCG on cv = 0.7943976926073789
lr = 0.1, mean nDCG on cv = 0.7512384755843294
lr = 1, mean nDCG on cv = 0.7183302160140588


In [96]:
#Число деревьев
n_estimators = [5, 10, 25, 50, 75, 100]
for n in n_estimators:
    model = xgb.XGBRanker(
                objective='rank:pairwise',
                n_estimators=n,
                learning_rate=1e-3,
                max_depth=1,
                verbosity=0
            )
    
    print("n_estimators = {}, mean nDCG on cv = {}".format(n, np.mean(cross_val_scores(model))))

n_estimators = 5, mean nDCG on cv = 0.8298192203072512
n_estimators = 10, mean nDCG on cv = 0.8192309947376692
n_estimators = 25, mean nDCG on cv = 0.8118661885529284
n_estimators = 50, mean nDCG on cv = 0.8060290045171883
n_estimators = 75, mean nDCG on cv = 0.805722399468667
n_estimators = 100, mean nDCG on cv = 0.805722399468667


In [98]:
#Регуляризация
reg_alpha = [1e-4, 1e-3, 1e-2, 1e-1, 1] # for L1 regularization
reg_lambda = [1e-4, 1e-3, 1e-2, 1e-1, 1] # for L2 regularization
for alpha in reg_alpha:
    for lamb in reg_lambda:
        model = xgb.XGBRanker(
                    objective='rank:pairwise',
                    n_estimators=5,
                    learning_rate=1e-3,
                    max_depth=1,
                    reg_alpha=alpha,
                    reg_lambda=lamb,
                    verbosity=0
                )
        
        print("alpha = {}, lambda = {}, mean nDCG on cv = {}".format(alpha, lamb, np.mean(cross_val_scores(model))))

alpha = 0.0001, lambda = 0.0001, mean nDCG on cv = 0.8298192203072512
alpha = 0.0001, lambda = 0.001, mean nDCG on cv = 0.8298192203072512
alpha = 0.0001, lambda = 0.01, mean nDCG on cv = 0.8298192203072512
alpha = 0.0001, lambda = 0.1, mean nDCG on cv = 0.8298192203072512
alpha = 0.0001, lambda = 1, mean nDCG on cv = 0.8298192203072512
alpha = 0.001, lambda = 0.0001, mean nDCG on cv = 0.8298192203072512
alpha = 0.001, lambda = 0.001, mean nDCG on cv = 0.8298192203072512
alpha = 0.001, lambda = 0.01, mean nDCG on cv = 0.8298192203072512
alpha = 0.001, lambda = 0.1, mean nDCG on cv = 0.8298192203072512
alpha = 0.001, lambda = 1, mean nDCG on cv = 0.8298192203072512
alpha = 0.01, lambda = 0.0001, mean nDCG on cv = 0.8298192203072512
alpha = 0.01, lambda = 0.001, mean nDCG on cv = 0.8298192203072512
alpha = 0.01, lambda = 0.01, mean nDCG on cv = 0.8298192203072512
alpha = 0.01, lambda = 0.1, mean nDCG on cv = 0.8298192203072512
alpha = 0.01, lambda = 1, mean nDCG on cv = 0.829819220307251

In [103]:
xgb_pairwise = xgb.XGBRanker(
                    objective='rank:pairwise',
                    n_estimators=5,
                    learning_rate=1e-3,
                    max_depth=1,
                    reg_alpha=1,
                    verbosity=0
                )

xgb_pairwise.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])],
                 y_train.relevance,
                 group=X_train.groupby('query_id').size().to_frame('size')['size'].to_numpy(),
                 verbose=0)

predictions = (X_test.groupby('query_id').apply(lambda x: predict(xgb_pairwise, x)))
xgb_pairwise_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG sore for XGBoost with rank:pairwise:', xgb_pairwise_ndcg)

nDCG sore for XGBoost with rank:pairwise: 0.831457281077564


#### rank:map

In [104]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8]
for depth in max_depths:
    model = xgb.XGBRanker(
                objective='rank:map',
                max_depth=depth
            )
    
    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores(model))))

max_depth = 1, mean nDCG on cv = 0.7252811898248768
max_depth = 2, mean nDCG on cv = 0.7204651021293488
max_depth = 3, mean nDCG on cv = 0.7168937165585063
max_depth = 4, mean nDCG on cv = 0.708219748791444
max_depth = 5, mean nDCG on cv = 0.706482853570288
max_depth = 6, mean nDCG on cv = 0.6943518604016933
max_depth = 7, mean nDCG on cv = 0.6905578011245271
max_depth = 8, mean nDCG on cv = 0.6831534026369166


In [107]:
#Скорость обучения
lrs = [1e-3, 1e-2, 1e-1, 1]
for lr in lrs:
    model = xgb.XGBRanker(
                objective='rank:map',
                learning_rate=lr,
                max_depth=1
            )
    
    print("lr = {}, mean nDCG on cv = {}".format(lr, np.mean(cross_val_scores(model))))

lr = 0.001, mean nDCG on cv = 0.8394773144097186
lr = 0.01, mean nDCG on cv = 0.8394773144097186
lr = 0.1, mean nDCG on cv = 0.7398893270801723
lr = 1, mean nDCG on cv = 0.7177144774666341


In [108]:
#Число деревьев
n_estimators = [5, 10, 25, 50, 75, 100]
for n in n_estimators:
    model = xgb.XGBRanker(
                objective='rank:map',
                n_estimators=n,
                learning_rate=1e-3,
                max_depth=1,
                verbosity=0
            )
    print("n_estimators = {}, mean nDCG on cv = {}".format(n, np.mean(cross_val_scores(model))))

n_estimators = 5, mean nDCG on cv = 0.8394773144097186
n_estimators = 10, mean nDCG on cv = 0.8394773144097186
n_estimators = 25, mean nDCG on cv = 0.8394773144097186
n_estimators = 50, mean nDCG on cv = 0.8394773144097186
n_estimators = 75, mean nDCG on cv = 0.8394773144097186
n_estimators = 100, mean nDCG on cv = 0.8394773144097186


In [109]:
#Регуляризация
reg_alpha = [1e-3, 1e-2, 1e-1, 1] # for L1 regularization
reg_lambda = [1e-3, 1e-2, 1e-1, 1] # for L2 regularization
for alpha in reg_alpha:
    for lamb in reg_lambda:
        model = xgb.XGBRanker(objective='rank:map',
                              n_estimators=5,
                              learning_rate=1e-3,
                              max_depth=1,
                              reg_alpha=alpha,
                              reg_lambda=lamb,
                              verbosity=0)
        
        print("alpha = {}, lambda = {}, mean nDCG on cv = {}".format(alpha, lamb, np.mean(cross_val_scores(model))))

alpha = 0.001, lambda = 0.001, mean nDCG on cv = 0.8394773144097186
alpha = 0.001, lambda = 0.01, mean nDCG on cv = 0.8394773144097186
alpha = 0.001, lambda = 0.1, mean nDCG on cv = 0.8394773144097186
alpha = 0.001, lambda = 1, mean nDCG on cv = 0.8394773144097186
alpha = 0.01, lambda = 0.001, mean nDCG on cv = 0.8394773144097186
alpha = 0.01, lambda = 0.01, mean nDCG on cv = 0.8394773144097186
alpha = 0.01, lambda = 0.1, mean nDCG on cv = 0.8394773144097186
alpha = 0.01, lambda = 1, mean nDCG on cv = 0.8394773144097186
alpha = 0.1, lambda = 0.001, mean nDCG on cv = 0.8394773144097186
alpha = 0.1, lambda = 0.01, mean nDCG on cv = 0.8394773144097186
alpha = 0.1, lambda = 0.1, mean nDCG on cv = 0.8394773144097186
alpha = 0.1, lambda = 1, mean nDCG on cv = 0.8394773144097186
alpha = 1, lambda = 0.001, mean nDCG on cv = 0.8394773144097186
alpha = 1, lambda = 0.01, mean nDCG on cv = 0.8394773144097186
alpha = 1, lambda = 0.1, mean nDCG on cv = 0.8394773144097186
alpha = 1, lambda = 1, mean 

In [120]:
xgb_map = xgb.XGBRanker(
                objective='rank:map',
                n_estimators=5,
                learning_rate=1e-3,
                max_depth=1,
                verbosity=0
            )

xgb_map.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])],
            y_train.relevance,
            group=X_train.groupby('query_id').size().to_frame('size')['size'].to_numpy(),
            verbose=0)

predictions = (X_test.groupby('query_id').apply(lambda x: predict(xgb_map, x)))
xgb_map_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG sore for XGBoost with rank:map:', xgb_map_ndcg)

nDCG sore for XGBoost with rank:map: 0.8562445567109971


#### rank:ndcg

In [113]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8,9,10]
for depth in max_depths:
    model = xgb.XGBRanker(
                objective='rank:ndcg',
                max_depth=depth
            )

    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores(model))))

max_depth = 1, mean nDCG on cv = 0.9060168475669513
max_depth = 2, mean nDCG on cv = 0.9060168475669513
max_depth = 3, mean nDCG on cv = 0.9060168475669513
max_depth = 4, mean nDCG on cv = 0.9060168475669513
max_depth = 5, mean nDCG on cv = 0.9060168475669513
max_depth = 6, mean nDCG on cv = 0.9060168475669513
max_depth = 7, mean nDCG on cv = 0.9060168475669513
max_depth = 8, mean nDCG on cv = 0.9060168475669513
max_depth = 9, mean nDCG on cv = 0.9060168475669513
max_depth = 10, mean nDCG on cv = 0.9060168475669513


In [125]:
#Скорость обучения
lrs = [1e-3, 1e-2, 1e-1, 1]
for lr in lrs:
    model = xgb.XGBRanker(
                objective='rank:ndcg',
                learning_rate=lr,
                max_depth=1
            )
    
    print("lr = {}, mean nDCG on cv = {}".format(lr, np.mean(cross_val_scores(model))))

lr = 0.001, mean nDCG on cv = 0.9060168475669513
lr = 0.01, mean nDCG on cv = 0.9060168475669513
lr = 0.1, mean nDCG on cv = 0.9060168475669513
lr = 1, mean nDCG on cv = 0.9060168475669513


In [126]:
#Число деревьев
n_estimators = [1, 2, 3, 5, 10, 25, 50, 75, 100]
for n in n_estimators:
    model = xgb.XGBRanker(
                objective='rank:ndcg',
                n_estimators=n,
                max_depth=1,
                verbosity=0
            )
    
    print("n_estimators = {}, mean nDCG on cv = {}".format(n, np.mean(cross_val_scores(model))))

n_estimators = 1, mean nDCG on cv = 0.9060168475669513
n_estimators = 2, mean nDCG on cv = 0.9060168475669513
n_estimators = 3, mean nDCG on cv = 0.9060168475669513
n_estimators = 5, mean nDCG on cv = 0.9060168475669513
n_estimators = 10, mean nDCG on cv = 0.9060168475669513
n_estimators = 25, mean nDCG on cv = 0.9060168475669513
n_estimators = 50, mean nDCG on cv = 0.9060168475669513
n_estimators = 75, mean nDCG on cv = 0.9060168475669513
n_estimators = 100, mean nDCG on cv = 0.9060168475669513


In [127]:
#Регуляризация
reg_alpha = [1e-3, 1e-2, 1e-1, 1] # for L1 regularization
reg_lambda = [1e-3, 1e-2, 1e-1, 1] # for L2 regularization
for alpha in reg_alpha:
    for lamb in reg_lambda:
        model = xgb.XGBRanker(
                    objective='rank:ndcg',
                    reg_alpha=alpha,
                    reg_lambda=lamb,
                    verbosity=0
                )
        
        print("alpha = {}, lambda = {}, mean nDCG on cv = {}".format(alpha, lamb, np.mean(cross_val_scores(model))))

alpha = 0.001, lambda = 0.001, mean nDCG on cv = 0.9060168475669513
alpha = 0.001, lambda = 0.01, mean nDCG on cv = 0.9060168475669513
alpha = 0.001, lambda = 0.1, mean nDCG on cv = 0.9060168475669513
alpha = 0.001, lambda = 1, mean nDCG on cv = 0.9060168475669513
alpha = 0.01, lambda = 0.001, mean nDCG on cv = 0.9060168475669513
alpha = 0.01, lambda = 0.01, mean nDCG on cv = 0.9060168475669513
alpha = 0.01, lambda = 0.1, mean nDCG on cv = 0.9060168475669513
alpha = 0.01, lambda = 1, mean nDCG on cv = 0.9060168475669513
alpha = 0.1, lambda = 0.001, mean nDCG on cv = 0.9060168475669513
alpha = 0.1, lambda = 0.01, mean nDCG on cv = 0.9060168475669513
alpha = 0.1, lambda = 0.1, mean nDCG on cv = 0.9060168475669513
alpha = 0.1, lambda = 1, mean nDCG on cv = 0.9060168475669513
alpha = 1, lambda = 0.001, mean nDCG on cv = 0.9060168475669513
alpha = 1, lambda = 0.01, mean nDCG on cv = 0.9060168475669513
alpha = 1, lambda = 0.1, mean nDCG on cv = 0.9060168475669513
alpha = 1, lambda = 1, mean 

In [133]:
xgb_ndcg = xgb.XGBRanker(
                objective='rank:ndcg'
            )

xgb_ndcg.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])],
            y_train.relevance,
            group=X_train.groupby('query_id').size().to_frame('size')['size'].to_numpy()
            )

predictions = (X_test.groupby('query_id').apply(lambda x: predict(xgb_ndcg, x)))
xgb_ndcg_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG score for XGBoost with rank:map:', xgb_ndcg_ndcg)

nDCG score for XGBoost with rank:map: 0.9113383867102114


### CatBoost 🐈 

In [43]:
import catboost as cat

#### YetiRank

In [84]:
def cross_val_scores_cat(model):
    ndcg_score = []
    for i in range(n_folds):
        (X_val_train, y_val_train), (X_val_test, y_val_test) = get_i_fold(new_train_data, i)
        
        train = cat.Pool(
            data=X_val_train.loc[:, ~X_val_train.columns.isin(['query_id'])],
            label=y_val_train.relevance,
            group_id=X_val_train['query_id']
        )
        
        model.fit(train, silent=True)

        predictions = (X_val_test.groupby('query_id').apply(lambda x: np.clip(predict(model, x), 0., 1.)))
        ndcg_score.append(nDCG_query(y_val_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions))
    
    return ndcg_score

In [46]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8,9,10]
for depth in max_depths:
    model = cat.CatBoostRanker(
                loss_function='YetiRank',
                depth=depth
            )
    
    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores_cat(model))))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

max_depth = 1, mean nDCG on cv = 0.7766081494867103
max_depth = 2, mean nDCG on cv = 0.7870632479608026
max_depth = 3, mean nDCG on cv = 0.784871556681154
max_depth = 4, mean nDCG on cv = 0.7765367455361624
max_depth = 5, mean nDCG on cv = 0.7698203441960811
max_depth = 6, mean nDCG on cv = 0.7578711393202677
max_depth = 7, mean nDCG on cv = 0.7468182137727795
max_depth = 8, mean nDCG on cv = 0.7309169702117039
max_depth = 9, mean nDCG on cv = 0.7259089057770216
max_depth = 10, mean nDCG on cv = 0.7188063032484985


In [51]:
%%capture

#Скорость обучения
lrs = [1e-3, 1e-2, 1e-1, 1]
for lr in lrs:
    model = cat.CatBoostRanker(
                loss_function='YetiRank',
                learning_rate=lr,
                depth=2
            )
    
    print("lr = {}, mean nDCG on cv = {}".format(lr, np.mean(cross_val_scores_cat(model))))

lr = 0.001, mean nDCG on cv = 0.7505595131221626
lr = 0.01, mean nDCG on cv = 0.7752885545231284
lr = 0.1, mean nDCG on cv = 0.7876547151056061
lr = 1, mean nDCG on cv = 0.7693628323613971


In [53]:
#Число деревьев
n_estimators = [1, 2, 3, 5, 10, 25, 50, 75, 100]
for n in n_estimators:
    model = cat.CatBoostRanker(
                loss_function='YetiRank',
                n_estimators=n,
                learning_rate=0.1,
                depth=2
            )
    
    print("n_estimators = {}, mean nDCG on cv = {}".format(n, np.mean(cross_val_scores_cat(model))))

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

n_estimators = 1, mean nDCG on cv = 0.8108161822289202
n_estimators = 2, mean nDCG on cv = 0.797531444301578
n_estimators = 3, mean nDCG on cv = 0.7754764982880058
n_estimators = 5, mean nDCG on cv = 0.7675406346888096
n_estimators = 10, mean nDCG on cv = 0.7618506824539405
n_estimators = 25, mean nDCG on cv = 0.7609130029414737
n_estimators = 50, mean nDCG on cv = 0.7724389032749455
n_estimators = 75, mean nDCG on cv = 0.7732299844771097
n_estimators = 100, mean nDCG on cv = 0.7786225470175856


In [47]:
#Регуляризация
reg_lambda = [1e-3, 1e-2, 1e-1, 1]
for lamb in reg_lambda:
    model = cat.CatBoostRanker(
                loss_function='YetiRank',
                n_estimators=1,
                learning_rate=0.1,
                depth=2,
                reg_lambda=lamb
            )
        
    print("lambda = {}, mean nDCG on cv = {}".format(lamb, np.mean(cross_val_scores_cat(model))))

lambda = 0.001, mean nDCG on cv = 0.8108161822289202
lambda = 0.01, mean nDCG on cv = 0.8108161822289202
lambda = 0.1, mean nDCG on cv = 0.8108161822289202
lambda = 1, mean nDCG on cv = 0.8108161822289202


In [56]:
cat_yetirank = cat.CatBoostRanker(
                    loss_function='YetiRank',
                    n_estimators=1,
                    learning_rate=0.1,
                    max_depth=2
                )

train = cat.Pool(
            data=X_train.loc[:, ~X_train.columns.isin(['query_id'])],
            label=y_train.relevance,
            group_id=X_train['query_id']
        )
        
cat_yetirank.fit(train, silent=True)

predictions = (X_test.groupby('query_id').apply(lambda x: np.clip(predict(cat_yetirank, x), 0., 1.)))
cat_yetirank_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG score for CatBoost with YetiRank:', cat_yetirank_ndcg)

nDCG score for CatBoost with YetiRank: 0.8012822139582864


nDCG на контроле 👇

In [130]:
print("XGB with linear:", xgb_linear_ndcg)
print("XGB with pairwise:", xgb_pairwise_ndcg)
print("XGB with map:", xgb_map_ndcg)
print("XGB with ndcg:", xgb_ndcg_ndcg)

XGB with linear: 0.9113383867102114
XGB with pairwise: 0.831457281077564
XGB with map: 0.8562445567109971
XGB with ndcg: 0.9113383867102114


In [60]:
print('CatBoost with YetiRank:', cat_yetirank_ndcg)

CatBoost with YetiRank: 0.8012822139582864


**Вопросы:**
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?
  - Для оценивания качества моделей используйте метрику nDCG на контроле.
  
**Выводы:**  
- Для данной задачи лучше всего подошел XGBoost c reg:linear и rank:ndcg функциями потерь. Для каждой функции потерь были подобраны гиперпараметры (n_estimators, learning_rate, max_depth, reg_alpha). Также я пыталась подобрать гиперпараметр reg_lambda, но это не помогло улучшить скор моделей :( Качество каждой полученной модели на контрольной выборке выписано выше ☝️  
- По полученным nDCG можно сделать вывод, что pointwise подход для данной задачи позволяет получить наилучшее качество нашей ранжирующей системы  
- Разница при использовании различных функций потерь очевидна невооруженным взглядом: nDCG изменяется от 0.801 до 0.911  

**XGB with linear**
- Достоинства: получилась модель с хорошим скором, довольно быстро посчиталось
- Недостатки: в pointwise подходе строится регрессия: для каждой отдельной пары запрос-документ необходимо предсказать её оценку. Почему это минус? Потому что мы не сравниваем релевантность документов относительно друг друга для запроса

**XGB with pairwise**
- Достоинства: довольно быстро обучилась моделька, функция потерь штрафует за то, что пара объектов неправильно упорядочена (плюс относительно первой модели с reg:linear)
- Недостатки: Получилось не самое лучшее качество :(

**XGB with map**
- Достоинства: listwise подход позволяет построить модель, на вход которой поступают сразу все документы, соответствующие запросу, а на выходе получается их перестановка
- Недостатки: подбор числа деревьев и реугляризации не позволил изменить nDCG

**XGB with ndcg**
- Достоинства: также позволяет на выходе получить перестановку документов для данного запроса
- Недостатки: подбор гиперпараметров не позволил изменить nDCG

**CatBoost with YetiRank**
- Достоинства: в 3 задании я добавила категориальные признаки (катбуст может работать с ними из коробки), это позволило повысить nDCG
- Недостатки: обучался дольше XGBoost, подбор параметров регуляризации не позволил изменить nDCG. Также я пыталась обучить катбуст с YetiRankPairwise, но модель обучалась оооочень долго и я не включила это в отчет



rank:map, rank:ndcg — реализация LambdaRank для двух метрик: MAP и nDCG. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как nDCG, нужно домножить градиент функционала  𝑂𝑏𝑗(𝑎)  на значение  Δ𝑁𝐷𝐶𝐺𝑖𝑗  — изменение значения функционала качества при замене  𝑥𝑖  на  𝑥𝑗 . Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса listwise моделей.

**(1 балл) Задание 3.** Одним из основных преимуществ CatBoost'a является обработка категориальных факторов «из коробки». Добавьте в датасет различные категориальные факторы из данных и обучите заново CatBoost модели. Улучшилось ли качество?

In [42]:
def get_language(x):
    try:
        language = langdetect.detect(x)
    except:
        language = "unknown"
    return language

In [43]:
query_langs = np.unique(train_data['query'].apply(lambda x: get_language(x)))
org_name_langs = np.unique(train_data['org_name'].apply(lambda x: get_language(x)))

In [44]:
langs = np.concatenate((query_langs, org_name_langs))
langs

array(['af', 'bg', 'ca', 'cs', 'cy', 'da', 'de', 'en', 'es', 'et', 'fi',
       'fr', 'hr', 'hu', 'id', 'it', 'lt', 'lv', 'mk', 'nl', 'no', 'pl',
       'pt', 'ro', 'ru', 'sk', 'sl', 'so', 'sv', 'sw', 'tl', 'tr', 'uk',
       'vi', 'af', 'bg', 'ca', 'cs', 'cy', 'da', 'de', 'en', 'es', 'et',
       'fi', 'fr', 'hr', 'hu', 'id', 'it', 'lt', 'lv', 'mk', 'nl', 'no',
       'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'so', 'sq', 'sv', 'sw', 'tl',
       'tr', 'uk', 'unknown', 'vi'], dtype=object)

In [45]:
qlang2id = {lang: idx for idx, lang in enumerate(np.unique(langs))}

In [46]:
df = pd.DataFrame()
df['query_id'] = train_data['query_id']
df['query_words_count'] = train_data['query'].apply(lambda query: len(getWordsFromString(query)))
df['org_name_words_count'] = train_data['org_name'].apply(lambda name: len(getWordsFromString(name)))
df['intersection_words_count'] = train_data[['query', 'org_name']].apply(getIntersectionWordsCount, axis=1)
df['jaccard_score'] = df['intersection_words_count'] / (df['query_words_count'] + df['org_name_words_count'] - df['intersection_words_count'])
df['synonymous_names_count'] = train_data['org_id'].apply(getSynonymousNamesCount)

df['rubrics_count'] = train_data['org_id'].apply(getRubricsCount)
df['worktime'] = train_data['org_id'].apply(getWorkTime)
df['workdays_count'] = train_data['org_id'].apply(getDaysCount)

df['query_lang'] = train_data['query'].apply(lambda x: qlang2id[get_language(x)]) #categorical
df['org_name_lang'] = train_data['org_name'].apply(lambda x: qlang2id[get_language(x)]) #categorical

In [57]:
df['lang_match'] = train_data[['query', 'org_name']].apply(lambda x: int(x[0] == x[1]), axis=1) #1/0

In [58]:
def get_org_geo_id(org_id) -> str:
    node = train_org_json.get(str(org_id))
    if node is not None:
        return str(node['address']['geo_id'])
    return 0

df['org_id_match'] = train_data[['org_id', 'region']].apply(lambda x: int(str(get_org_geo_id(x[0])) == str(x[1])), axis=1) #1/0

In [59]:
def dist1(x1, x2) -> float:
    return abs(x1[0] - x2[0]) + abs(x1[1] - x2[1])

def dist2(x1, x2) -> float:
    return np.sqrt((x1[0] - x2[0]) ** 2 + (x1[1] - x2[1]) ** 2)

def get_org_geo_coord(org_id):
    node = train_org_json.get(str(org_id))
    if node is not None:
        return node['address']['pos']['coordinates']
    return [0, 0]

In [60]:
df['dist1'] = train_data[['window_center', 'org_id']].apply(lambda x: dist1(get_org_geo_coord(x[1]), list(map(float, x[0].split(',')))), axis=1)
df['dist2'] = train_data[['window_center', 'org_id']].apply(lambda x: dist2(get_org_geo_coord(x[1]), list(map(float, x[0].split(',')))), axis=1) #1/0

In [61]:
df['relevance'] = train_data['relevance']
df.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count,query_lang,org_name_lang,lang_match,org_id_match,dist1,dist2,relevance
0,11,7,7,1,0.076923,5,1,240.0,1,24,33,0,0,0.490343,0.443272,0.0
1,11,7,7,1,0.076923,5,1,540.0,5,24,33,0,0,0.469583,0.399923,0.0
2,11,7,4,1,0.100000,7,1,480.0,5,24,33,0,0,0.476728,0.435877,0.0
3,11,7,4,1,0.100000,7,1,240.0,1,24,33,0,0,0.432932,0.397661,0.0
4,11,7,2,1,0.125000,3,1,540.0,5,24,24,0,0,0.490446,0.443338,0.0


In [62]:
cat = ['query_lang', 'org_name_lang', 'lang_match', 'org_id_match']

df[col] = df[col].astype('category')

In [63]:
new_train_data = df.iloc[X_train_inds]
X_train = new_train_data.loc[:, ~new_train_data.columns.isin(['relevance'])]
y_train = new_train_data.loc[:, new_train_data.columns.isin(['query_id', 'relevance'])]

new_test_data = df.iloc[X_test_inds]

X_test = new_test_data.loc[:, ~new_test_data.columns.isin(['relevance'])]
y_test = new_test_data.loc[:, new_test_data.columns.isin(['query_id', 'relevance'])]

In [64]:
X_train.head()

,query_id,query_words_count,org_name_words_count,intersection_words_count,jaccard_score,synonymous_names_count,rubrics_count,worktime,workdays_count,query_lang,org_name_lang,lang_match,org_id_match,dist1,dist2
0,11,7,7,1,0.076923,5,1,240.0,1,24,33,0,0,0.490343,0.443272
1,11,7,7,1,0.076923,5,1,540.0,5,24,33,0,0,0.469583,0.399923
2,11,7,4,1,0.100000,7,1,480.0,5,24,33,0,0,0.476728,0.435877
3,11,7,4,1,0.100000,7,1,240.0,1,24,33,0,0,0.432932,0.397661
4,11,7,2,1,0.125000,3,1,540.0,5,24,24,0,0,0.490446,0.443338


In [82]:
import catboost as cat

Выше я добавила категориальные признаки, теперь попробуем обучить катбуст на старых + новодобавленных признаках

In [85]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8,9,10]
for depth in max_depths:
    model = cat.CatBoostRanker(
        loss_function='YetiRank',
        depth=depth
    )
    
    print("max_depth = {}, mean nDCG on cv = {}".format(depth, np.mean(cross_val_scores_cat(model))))

max_depth = 1, mean nDCG on cv = 0.7870646088773717
max_depth = 2, mean nDCG on cv = 0.790459199858747
max_depth = 3, mean nDCG on cv = 0.7913356727218813
max_depth = 4, mean nDCG on cv = 0.7883086670742416
max_depth = 5, mean nDCG on cv = 0.7836831507673894
max_depth = 6, mean nDCG on cv = 0.7779308865421689
max_depth = 7, mean nDCG on cv = 0.7730753617898072
max_depth = 8, mean nDCG on cv = 0.7697731767406019
max_depth = 9, mean nDCG on cv = 0.7682415446781499
max_depth = 10, mean nDCG on cv = 0.7638110560199193


In [87]:
#Число деревьев
n_estimators = [1, 2, 3, 5, 10, 25, 50, 75, 100]
for n in n_estimators:
    model = cat.CatBoostRanker(
        loss_function='YetiRank',
        n_estimators=n,
        depth=3
    )
    
    print("n_estimators = {}, mean nDCG on cv = {}".format(n, np.mean(cross_val_scores_cat(model))))

n_estimators = 1, mean nDCG on cv = 0.7866286448369745
n_estimators = 2, mean nDCG on cv = 0.7656187760885793
n_estimators = 3, mean nDCG on cv = 0.773063916342781
n_estimators = 5, mean nDCG on cv = 0.7613294414586772
n_estimators = 10, mean nDCG on cv = 0.7578869217727137
n_estimators = 25, mean nDCG on cv = 0.7541979460006166
n_estimators = 50, mean nDCG on cv = 0.7577001355228449
n_estimators = 75, mean nDCG on cv = 0.7659831059723373
n_estimators = 100, mean nDCG on cv = 0.7716402705348906


In [88]:
#Регуляризация
reg_lambda = [1e-3, 1e-2, 1e-1, 1]
for lamb in reg_lambda:
    model = cat.CatBoostRanker(
        loss_function='YetiRank',
        n_estimators=1,
        depth=2,
        reg_lambda=lamb
    )
        
    print("lambda = {}, mean nDCG on cv = {}".format(lamb, np.mean(cross_val_scores_cat(model))))

lambda = 0.001, mean nDCG on cv = 0.8090345830021708
lambda = 0.01, mean nDCG on cv = 0.8090345830021708
lambda = 0.1, mean nDCG on cv = 0.8090345830021708
lambda = 1, mean nDCG on cv = 0.8090345830021708


In [89]:
#Максимальная глубина
max_depths = [1,2,3,4,5,6,7,8,9,10]
#Число деревьев
n_estimators = [1, 2, 3, 5, 10, 25, 50, 75, 100]

for depth in max_depths:
    for n in n_estimators:
        model = cat.CatBoostRanker(
                    loss_function='YetiRank',
                    n_estimators=n,
                    depth=depth,
                    reg_lambda=0.1
                )
    
        print("max_depth = {}, n_estimators = {}, mean nDCG on cv = {}".format(depth, n, np.mean(cross_val_scores_cat(model))))

max_depth = 1, n_estimators = 1, mean nDCG on cv = 0.8359569802215141
max_depth = 1, n_estimators = 2, mean nDCG on cv = 0.8289812056174763
max_depth = 1, n_estimators = 3, mean nDCG on cv = 0.8147529098037755
max_depth = 1, n_estimators = 5, mean nDCG on cv = 0.8069605294541523
max_depth = 1, n_estimators = 10, mean nDCG on cv = 0.7991514774378332
max_depth = 1, n_estimators = 25, mean nDCG on cv = 0.7919795622042478
max_depth = 1, n_estimators = 50, mean nDCG on cv = 0.7752708864378409
max_depth = 1, n_estimators = 75, mean nDCG on cv = 0.7683336527584208
max_depth = 1, n_estimators = 100, mean nDCG on cv = 0.7665664139192264
max_depth = 2, n_estimators = 1, mean nDCG on cv = 0.8090345830021708
max_depth = 2, n_estimators = 2, mean nDCG on cv = 0.8044876196909293
max_depth = 2, n_estimators = 3, mean nDCG on cv = 0.7933973796415991
max_depth = 2, n_estimators = 5, mean nDCG on cv = 0.7938736845689196
max_depth = 2, n_estimators = 10, mean nDCG on cv = 0.7784434480176519
max_depth = 2

In [93]:
cat_yetirank_with_categorical = cat.CatBoostRanker(
        loss_function='YetiRank',
        n_estimators=1,
        depth=1,
        reg_lambda=0.1
    )

train = cat.Pool(
            data=X_train.loc[:, ~X_train.columns.isin(['query_id'])],
            label=y_train.relevance,
            group_id=X_train['query_id']
        )
        
cat_yetirank_with_categorical.fit(train, silent=True)

predictions = (X_test.groupby('query_id').apply(lambda x: np.clip(predict(cat_yetirank_with_categorical, x), 0., 1.)))
cat_yetirank_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)

print('nDCG score for CatBoost with YetiRank with categorical features:', cat_yetirank_ndcg)

nDCG score for CatBoost with YetiRank with categorical features: 0.8338838386543777


Добавление категориальных фичей в катбуст позволило повысить nDCG с 0.8013 до 0.834. Возможно, стоит добавить ещё категориальных фичей или погенерить другие, чтобы увеличить nDCG ещё.

#### Пользовательская функция потерь

Библиотека XGBoost позволяет использовать пользовательские функции потерь. Для этого необходимо реализовать функцию, принимающую на вход вектор предсказанных значений и обучающую выборку, и возвращающую градиент и гессиан, посчитанный по входным данным.

Важно отметить, что XGBoost использует диагональную аппроксимацию гессиана, таким образом все недиагональные элементы считаются малозначимыми и приравниваются нулю, поэтому и градиент, и гессиан являются векторами длины размера обучающей выборки.

**(1 балла) Задание 4.** Реализуйте экспоненциальную функцию потерь для XGBoost:
$$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = e^{-M} $$

Обучите модель с помощью данной функции потерь, настройте параметры.

**Комментарии к реализации**

В случае ранжирования XGBoost'у необходимо знать о разбиении всех объектов на группы. В нашем случае в одну группу будут входить документы, соответствующие одному запросу. Функция, считающая градиент и гессиан по данным, должна знать данное разбиение датасета. Однако питоновский интерфейс класса *DMatrix* (в котором хранится датасет) не дает возможности получить это разбиение. В этом случае нужно реализовать функцию потерь в качестве функтора, конструктор которого принимает разбиение на группы в качестве параметра.

Пример реализации своей функции потерь можно найти [в официальной справке](https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html#customized-objective-function) 

In [65]:
import xgboost as xgb

In [82]:
class ExponentialPairwiseLoss(object):
    def __init__(self, groups):
        self.groups = groups
                        
    def __call__(self, pred, dtrain):
        start_group_idx = np.concatenate(([0], np.cumsum(self.groups)))
        grad = np.zeros_like(pred)
        hess = np.zeros_like(pred)
        
        for gr_idx, group in enumerate(self.groups):
            for i in range(start_group_idx[gr_idx], start_group_idx[gr_idx + 1]):
                for j in range(i + 1, start_group_idx[gr_idx + 1]):
                    value = 0
                    if dtrain[i] < dtrain[j]:
                        value = np.exp(pred[i] - pred[j])
                        grad[i] += value
                        grad[j] -= value
                    elif dtrain[i] > dtrain[j]:
                        value = np.exp(pred[j] - pred[i])
                        grad[j] += value
                        grad[i] -= value
                    hess[i] += value
                    hess[j] += value

        return grad, hess  

In [85]:
model = xgb.XGBRegressor(objective='reg:linear', verbosity=0)

model.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])], y_train.relevance)

predictions = (X_test.groupby('query_id').apply(lambda x: predict(model, x)))
xgb_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG score for XGBoost with reg:linear:', xgb_ndcg)

nDCG score for XGBoost with reg:linear: 0.7103656969905512


In [84]:
groups = X_train.groupby('query_id').size().to_frame('size')['size'].to_numpy()

model = xgb.XGBRegressor(objective=ExponentialPairwiseLoss(groups))

model.fit(X_train.loc[:, ~X_train.columns.isin(['query_id'])], y_train.relevance, verbose=0)

predictions = (X_test.groupby('query_id').apply(lambda x: predict(model, x)))
xgb_ndcg = nDCG_query(y_test.groupby('query_id').apply(lambda x: list(x.relevance)), predictions)
print('nDCG score for XGBoost with ExponentialPairwiseLoss:', xgb_ndcg)

nDCG score for XGBoost with ExponentialPairwiseLoss: 0.9113383867102114


ExponentialPairwiseLoss увеличивает nDCG по сравнению с reg:linear